Paper FSMol (https://openreview.net/pdf?id=701FtuyLlAd) doesn't go over all evaluations. Below are results after I finished my local testing 

In [17]:
import glob
import numpy as np
import pandas as pd 

def read_result(filename, experimentid):
  df = pd.read_csv(filename)
  
  vals = [float(i.split('+/-')[0]) for i in df[experimentid] if isinstance(i, str)]
  return np.mean(vals)

path = '/home/jim/Documents/FS-Mol/baselines/'
filenames = glob.glob(path+"*.csv")
experimentids = ['16_train', '32_train', '64_train', '128_train', '256_train']

df = {'expname': []}
for eid in experimentids: df[eid] = []
for filename in filenames:
  df['expname'].append(filename.split('/')[-1])
  for experimentid in experimentids:
    df[experimentid].append(read_result(filename, experimentid))
df = pd.DataFrame(df)
df.sort_values("16_train", inplace=True)
df 



,expname,16_train,32_train,64_train,128_train,256_train
0,GNN-ST_summary.csv,0.495732,0.501739,0.505497,0.526764,0.501907
6,kNN_summary.csv,0.525879,0.550599,0.577051,0.614293,0.618977
4,MAT_summary.csv,0.527070,0.544490,0.567389,0.611146,0.590884
3,random_forest_summary.csv,0.568408,0.600204,0.638261,0.687904,0.705535
2,GNN-Multitask_summary.csv,0.587274,0.618943,0.651720,0.697828,0.682837
1,MAML-Support16_summary.csv,0.635108,0.642057,0.648580,0.667299,0.631070
5,ProtoNet-gnn+ecfp+fc-Support64_summary.csv,0.680968,0.717338,0.745688,0.776376,0.735860


In [15]:
import glob 
import warnings 

import numpy as np 
import pandas as pd 

warnings.filterwarnings("ignore")

def read_result(filename, num_train_requested, metric):
  df = pd.read_csv(filename)
  val = [float(v) for i,v in zip(df['num_train_requested'], df[metric]) if i==num_train_requested]
  return np.mean(val)

path = '/home/jim/Documents/FS-Mol/outputs/'
folders = ['FSMol_Eval_kNN_2023-06-04_16-54-30', 'FSMol_Eval_randomForest_2023-06-04_16-37-02', 'FSMol_Eval_ProtoNet_2023-06-04_23-34-33']
num_train_requested = [16, 32, 64, 128]
metrics = ['num_train_requested', 'num_train', 'fraction_positive_train', 'num_test', 'fraction_positive_test', 'seed', 'valid_score', 'average_precision_score', 'roc_auc', 'acc', 'balanced_acc', 'precision', 'recall', 'f1_score', 'delta_auprc']

df = {'method': [], '#': []}
for metric in metrics: df[metric] = []

for method in folders:
  filenames = glob.glob(path+f"{method}/"+"*.csv")
  method = method.split('_')[2]
  for num_train in num_train_requested:
    df['method'].append(method)
    df['#'].append(str(num_train))

    for metric in metrics:
      val = [read_result(f, num_train, metric) for f in filenames]
      df[metric].append(f"{np.mean(val):.3f}+/-{np.std(val):.3f}" if val else "")
df = pd.DataFrame(df)
df
      


,method,#,num_train_requested,num_train,fraction_positive_train,num_test,fraction_positive_test,seed,valid_score,average_precision_score,roc_auc,acc,balanced_acc,precision,recall,f1_score,delta_auprc
0,kNN,16,16.000+/-0.000,16.000+/-0.000,0.476+/-0.050,342.089+/-506.874,0.471+/-0.046,4.500+/-0.000,nan+/-nan,0.526+/-0.075,0.569+/-0.069,0.544+/-0.052,0.546+/-0.051,0.513+/-0.063,0.704+/-0.120,0.579+/-0.081,0.055+/-0.056
1,kNN,32,32.000+/-0.000,32.000+/-0.000,0.472+/-0.048,326.089+/-506.874,0.472+/-0.046,4.500+/-0.000,nan+/-nan,0.551+/-0.083,0.598+/-0.080,0.567+/-0.062,0.569+/-0.061,0.531+/-0.068,0.713+/-0.117,0.600+/-0.080,0.079+/-0.069
2,kNN,64,64.000+/-0.000,64.000+/-0.000,0.473+/-0.047,294.089+/-506.874,0.471+/-0.046,4.500+/-0.000,nan+/-nan,0.577+/-0.093,0.628+/-0.090,0.589+/-0.071,0.591+/-0.070,0.550+/-0.075,0.730+/-0.103,0.621+/-0.075,0.106+/-0.082
3,kNN,128,128.000+/-0.000,128.000+/-0.000,0.473+/-0.046,230.089+/-506.874,0.467+/-0.046,4.500+/-0.000,nan+/-nan,0.614+/-0.097,0.662+/-0.097,0.612+/-0.078,0.616+/-0.077,0.567+/-0.078,0.755+/-0.101,0.642+/-0.077,0.147+/-0.092
4,randomForest,16,16.000+/-0.000,16.000+/-0.000,0.500+/-0.000,216.200+/-95.585,0.496+/-0.005,4.500+/-0.000,nan+/-nan,0.677+/-0.048,0.689+/-0.050,0.623+/-0.037,0.624+/-0.036,0.622+/-0.050,0.677+/-0.051,0.630+/-0.034,0.181+/-0.047
5,randomForest,32,32.000+/-0.000,32.000+/-0.000,0.500+/-0.000,200.200+/-95.585,0.495+/-0.006,4.500+/-0.000,nan+/-nan,0.708+/-0.055,0.726+/-0.058,0.657+/-0.041,0.658+/-0.040,0.659+/-0.052,0.667+/-0.039,0.651+/-0.038,0.213+/-0.054
6,randomForest,64,64.000+/-0.000,64.000+/-0.000,0.494+/-0.008,168.200+/-95.585,0.498+/-0.003,4.500+/-0.000,nan+/-nan,0.753+/-0.068,0.772+/-0.061,0.699+/-0.038,0.699+/-0.038,0.704+/-0.044,0.703+/-0.080,0.694+/-0.050,0.256+/-0.068
7,randomForest,128,128.000+/-0.000,128.000+/-0.000,0.497+/-0.004,104.200+/-95.585,0.490+/-0.015,4.500+/-0.000,nan+/-nan,0.777+/-0.081,0.793+/-0.065,0.708+/-0.048,0.707+/-0.048,0.698+/-0.053,0.727+/-0.066,0.706+/-0.054,0.287+/-0.087
8,ProtoNet,16,16.000+/-0.000,16.000+/-0.000,0.476+/-0.050,342.089+/-506.874,0.471+/-0.046,4.500+/-0.000,nan+/-nan,0.681+/-0.119,0.709+/-0.110,0.659+/-0.085,0.653+/-0.086,0.640+/-0.100,0.632+/-0.115,0.628+/-0.105,0.210+/-0.111
9,ProtoNet,32,32.000+/-0.000,32.000+/-0.000,0.472+/-0.048,326.089+/-506.874,0.472+/-0.046,4.500+/-0.000,nan+/-nan,0.717+/-0.123,0.743+/-0.112,0.688+/-0.091,0.684+/-0.092,0.667+/-0.105,0.669+/-0.112,0.664+/-0.107,0.246+/-0.116
